# Developing a chatbot using own documents with masking methods

In [1]:
# pip install langchain faiss-cpu sentence-transformers

## Problem Statement

##### We want to mask some fields while using LLMs like OpenAI, PaLM, Cloude etc.
##### When developing chatbot applications, confidential information is crucial for customer privacy. So if you want to use cloud-based LLMs like GPT4 (OpenAI), you need to find a solution for keeping those fields before sending them to the LLM provider.

#### We will use regular expressions to implement this idea for fields like phone number, email, credit card number, etc.
#### There are 4 steps to solve the problem
1. Read documents and split them into little chunks (LangChain - Document loaders)
1. Get embeddings of the chunks (Sentence transformers)
1. Create vector store to search relevant documents (LangChain - FAISS)
1. Generate response with LLMs. (LangChain - LLM chain)

### Import libs

In [2]:
import os
import openai
from langchain.prompts import (
    ChatPromptTemplate, 
    MessagesPlaceholder, 
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate
)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.chains import ConversationChain, LLMChain
from langchain.chat_models import ChatOpenAI, AzureChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
from langchain.callbacks import get_openai_callback
import re

os.environ["OPENAI_API_KEY"] = "sk-..."



### Lets build masking functions
#### First, we use regex to find specific fields in text. To implement masking logic, we will focus on 4 entities:
1. email
1. url
1. phone number
1. credit card number

Use can use other methods to improve this logic. For example, you can use named entity recognition to find names or locations.

In [4]:
def mask_entities(text, regex, mask_template, **kwargs):
    entity_regex_result = re.findall(regex, text)
    entity_mask_map = {}
    prefix = kwargs.get("prefix", "")
    for i, entity in enumerate(entity_regex_result):
        if entity not in entity_mask_map:
            mask_key = f"[{mask_template}_{prefix}_{i}]"
            entity_mask_map[entity] = mask_key
            text = text.replace(entity, mask_key)
    return text, entity_mask_map

def pre_masking(text, entity_data, **kwargs):
    text = text.replace("İ", "i").lower()
    masked_entities = {}
    prefix = kwargs.get("prefix", "")
    for entity_name, entity_regex, mask_template in entity_data:
        text, entity_mask_map = mask_entities(text, entity_regex, mask_template, prefix=prefix)
        masked_entities[entity_name] = entity_mask_map
    return text, masked_entities

def post_masking(text, masked_entities):
    for entity_map in masked_entities.values():
        for entity, mask in entity_map.items():
            text = text.replace(mask, entity)
    return text

# Define entity data: (name, regex, mask_template)
entity_data = [
    ("email", r'[\w\.-]+@[\w\.-]+', "EMAIL_MASK"),
    ("url", r'http\S+', "URL_MASK"),
    ("phone_number", r'\d{4}\s\d{3}\s\d{2}\s\d{2}|\d{4}\s\d{3}\s\d{4}|\d\s\d{3}\s\d{3}\s\d\s\d{3}|\d\s\d{3}\s\d{3}\s\d{2}\s\d{2}', "PHONE_NUMBER_MASK"),
    ("credit_card", r'\d{4}\s\d{4}\s\d{4}\s\d{4}', "CREDIT_CARD_MASK"), # 1234 1234 1234 1234 it can updated
]


### Test our functions with example data

In [5]:
# Example usage:
input_text = "Please 1234 1234 1234 1234 contact abc@abc.com or visit http://www.google.com"
print(input_text)
print("*"*50)
masked_text, masked_entities = pre_masking(input_text, entity_data, prefix="test")
print(masked_text)
print("*"*50)
print(masked_entities)
print("*"*50)

# Process the masked text...
final_text = post_masking(masked_text, masked_entities)
print(final_text)

Please 1234 1234 1234 1234 contact abc@abc.com or visit http://www.google.com
**************************************************
please [CREDIT_CARD_MASK_test_0] contact [EMAIL_MASK_test_0] or visit [URL_MASK_test_0]
**************************************************
{'email': {'abc@abc.com': '[EMAIL_MASK_test_0]'}, 'url': {'http://www.google.com': '[URL_MASK_test_0]'}, 'phone_number': {}, 'credit_card': {'1234 1234 1234 1234': '[CREDIT_CARD_MASK_test_0]'}}
**************************************************
please 1234 1234 1234 1234 contact abc@abc.com or visit http://www.google.com


In [6]:
# Example usage:
input_text = """Tel: 0 212 331 0 200
Fax: 0 212 332 18 93
Kep Adresi: dsm@hs02.kep.tr
Trendyol Sigorta iletişim bilgilerine nasıl ulaşabilirim?
Tüm soru ve taleplerin için 0850 955 14 14 numaralı Trendyol Sigorta Müşteri Hizmetleri'ni arayarak destek alabilirsin.
Çalışma Saatlerimiz: Pazartesi-Cumartesi 09:00-18:00
"""
print(input_text)
print("*"*50)
masked_text, masked_entities = pre_masking(input_text, entity_data, prefix="test")
print(masked_text)
print("*"*50)
print(masked_entities)
print("*"*50)

# Process the masked text...
final_text = post_masking(masked_text, masked_entities)
print(final_text)

Tel: 0 212 331 0 200
Fax: 0 212 332 18 93
Kep Adresi: dsm@hs02.kep.tr
Trendyol Sigorta iletişim bilgilerine nasıl ulaşabilirim?
Tüm soru ve taleplerin için 0850 955 14 14 numaralı Trendyol Sigorta Müşteri Hizmetleri'ni arayarak destek alabilirsin.
Çalışma Saatlerimiz: Pazartesi-Cumartesi 09:00-18:00

**************************************************
tel: [PHONE_NUMBER_MASK_test_0]
fax: [PHONE_NUMBER_MASK_test_1]
kep adresi: [EMAIL_MASK_test_0]
trendyol sigorta iletişim bilgilerine nasıl ulaşabilirim?
tüm soru ve taleplerin için [PHONE_NUMBER_MASK_test_2] numaralı trendyol sigorta müşteri hizmetleri'ni arayarak destek alabilirsin.
çalışma saatlerimiz: pazartesi-cumartesi 09:00-18:00

**************************************************
{'email': {'dsm@hs02.kep.tr': '[EMAIL_MASK_test_0]'}, 'url': {}, 'phone_number': {'0 212 331 0 200': '[PHONE_NUMBER_MASK_test_0]', '0 212 332 18 93': '[PHONE_NUMBER_MASK_test_1]', '0850 955 14 14': '[PHONE_NUMBER_MASK_test_2]'}, 'credit_card': {}}
********

### Load document and split

In [ ]:
# load document
with open('./trendyol-faq-sample.txt') as f:
    faq_sample = f.read()
faq_sample.splitlines()[:5]

['Ücret iadem ne zaman yapılır?',
 '1. İptal ettiğiniz ürünün ücret iadesi bankanıza bağlı olarak değişkenlik gösterebilir. Bu süre yaklaşık 1 haftayı bulabilir.',
 '',
 '2. İade ettiğiniz ürünün ücret iade süreci aşağıdaki gibidir;',
 '']

In [7]:
# init text splitter for creating chunks
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 200,
    chunk_overlap  = 30,
    length_function = len,
    add_start_index = True,
)

In [8]:
# create documents (chunks)
text_documents = text_splitter.create_documents([faq_sample])
print(text_documents[0])
print(text_documents[1])

page_content='Ücret iadem ne zaman yapılır?\n1. İptal ettiğiniz ürünün ücret iadesi bankanıza bağlı olarak değişkenlik gösterebilir. Bu süre yaklaşık 1 haftayı bulabilir.' metadata={'start_index': 0}
page_content='2. İade ettiğiniz ürünün ücret iade süreci aşağıdaki gibidir;\n\n• Ürün satıcıya ulaştıktan sonra en geç 48 saat içerisinde iade şartlarına uygunluğu kontrol edilir.' metadata={'start_index': 157}


### Load embedding model and embed documents

In [9]:
sentence_transformer_model = "paraphrase-multilingual-MiniLM-L12-v2"
embedding_model = SentenceTransformerEmbeddings(model_name=sentence_transformer_model)

/opt/homebrew/Caskroom/miniforge/base/envs/pl_test/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
texts = [d.page_content for d in text_documents]
metadatas_fields = [d.metadata for d in text_documents]
text_embeddings = embedding_model.embed_documents(texts) # embed all documents

In [11]:
len(text_embeddings), len(texts), len(metadatas_fields)

(138, 138, 138)

In [12]:
text_embedding_pairs = list(zip(texts, text_embeddings)) # create pairs for indexing

### Create vector store with FAISS backend

In [13]:
db = FAISS.from_embeddings(text_embedding_pairs, embedding_model, metadatas=metadatas_fields)

#### Check similar documents with exampe query

In [14]:
query = "trendyol sigorta müşteri hizmetleri"
docs = db.similarity_search_with_score(query, k=3)
docs

[(Document(page_content="Şikayetinle ilgili destek almak ve deneyimine dair görüşlerini paylaşmak için 0850 955 14 14 numaralı Trendyol Sigorta Müşteri Hizmetleri'ni arayarak bize ulaşabilirsin. Süreçlerimizi geliştirirken", metadata={'start_index': 17905}),
  10.569565),
 (Document(page_content='Trendyol Sigorta nasıl kullanılır?\n1) Sigorta ürünlerimizi Trendyol uygulamasından kolayca satın alabilirsin:', metadata={'start_index': 16845}),
  10.714605),
 (Document(page_content="- Trendyol'dan bir ürün satın alırken ürün detay sayfasında Ek Hizmetler başlığı altında sigorta tekliflerimizi inceleyebilirsin. Sepetine sigortalamak istediğin ürünle birlikte sigortayı da ekleyip", metadata={'start_index': 16955}),
  11.297446)]

In [15]:
db.save_local("faiss_index") # persist index to disk

### Create retreiver to find relevant documents

In [16]:
retriever = db.as_retriever(search_kwargs={"k": 3})
docs = retriever.get_relevant_documents(query)
docs

[Document(page_content="Şikayetinle ilgili destek almak ve deneyimine dair görüşlerini paylaşmak için 0850 955 14 14 numaralı Trendyol Sigorta Müşteri Hizmetleri'ni arayarak bize ulaşabilirsin. Süreçlerimizi geliştirirken", metadata={'start_index': 17905}),
 Document(page_content='Trendyol Sigorta nasıl kullanılır?\n1) Sigorta ürünlerimizi Trendyol uygulamasından kolayca satın alabilirsin:', metadata={'start_index': 16845}),
 Document(page_content="- Trendyol'dan bir ürün satın alırken ürün detay sayfasında Ek Hizmetler başlığı altında sigorta tekliflerimizi inceleyebilirsin. Sepetine sigortalamak istediğin ürünle birlikte sigortayı da ekleyip", metadata={'start_index': 16955})]

### Define system prompt

In [17]:
SYSTEM_PROMPT = """Bir eticaret firması için geliştirilmiş yardımcı ve nazik bir chatbotsun. 
'''RELEVANT_DOCS''' içerisindeki ifadelerin bazıları maskelenmiş olabilir. Cevap verirken bu alanları olduğu gibi bırak ve değişiklik yapma. Eğer maskelenmiş alan yoksa NONE değerini alır. Maskelenmiş alanlar şunlardan oluşuyor:
{masked_fields}

Cevap verirken yorum yapma ve sadece '''RELEVANT_DOCS''' içerisinde yer alan değerlere göre cevap ver. Verilen bilgilere göre cevaplayamadığın bir konuysa sadece '''UNK''' değeri ile cevap ver.
'''UNK''': 'Cevabı henüz bilmiyorum. Sana başka nasıl yardımcı olabilirim?'

'''RELEVANT_DOCS''':
{relevant_docs}

"""

### Create prompt templates

Prompt templates are pre-defined recipes for generating prompts for language models.

A template may include instructions, few-shot examples, and specific context and questions appropriate for a given task.

LangChain provides tooling to create and work with prompt templates.

LangChain strives to create model agnostic templates to make it easy to reuse existing templates across different language models.
[For more info](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/)

In [18]:
prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(SYSTEM_PROMPT, input_variables=["relevant_docs", "masked_fields"]),
        MessagesPlaceholder(variable_name="history"),
        HumanMessagePromptTemplate.from_template("User message: {query}", input_variables=["query"])
    ])

### Define LLM
We will use OpenAI gpt3.5-turbo

In [19]:
llm = ChatOpenAI(temperature=0, max_retries=2)

### Create memory window
We will store past message for follow up questions. [For more info](https://python.langchain.com/docs/modules/memory/types/buffer_window)

In [20]:
memory = ConversationBufferWindowMemory(memory_key="history", k=2, input_key="query", return_messages=True)

### Put all together as llm chain

In [21]:
llm_chain = LLMChain(
    prompt=prompt,
    memory=memory,
    llm=llm,
    verbose=True,
)

### Create chat function to apply all logic

In [22]:
def chat(user_message):
    docs = retriever.get_relevant_documents(user_message) # get relevant documents
    page_metadata = [doc.metadata for doc in docs] # get metadata of documents

    masked_keys = set()
    relevant_docs = "\n".join([doc.page_content for doc in docs]) # join documents
    masked_content, masked_fields = pre_masking(relevant_docs, entity_data) # mask entities

    # get unique masked keys for prompt
    for field_key, field_val in masked_fields.items():
        masked_keys.update(field_val.values())   

    result = {}
    with get_openai_callback() as cb:
        # predict
        result["gpt_raw_response"] = llm_chain.predict(query=user_message, 
                                                relevant_docs=masked_content, 
                                                masked_fields="\n".join(list(masked_keys)))
        # postprocess
        result["parsed_response"] = post_masking(result["gpt_raw_response"], masked_fields)

        # get callback info
        result["total_tokens"] = cb.total_tokens
        result["prompt_tokens"] = cb.prompt_tokens
        result["completion_tokens"] = cb.completion_tokens
        result["total_cost"] = cb.total_cost
        result["page_metadata"] = page_metadata

    return result

### Chat with our documents

In [23]:
response = chat("trendyol sigortayla ilgili sorularım için kime ulaşabilirim?")



> Entering new LLMChain chain...
Prompt after formatting:
System: Bir eticaret firması için geliştirilmiş yardımcı ve nazik bir chatbotsun. 
'''RELEVANT_DOCS''' içerisindeki ifadelerin bazıları maskelenmiş olabilir. Cevap verirken bu alanları olduğu gibi bırak ve değişiklik yapma. Eğer maskelenmiş alan yoksa NONE değerini alır. Maskelenmiş alanlar şunlardan oluşuyor:
[PHONE_NUMBER_MASK__0]

Cevap verirken yorum yapma ve sadece '''RELEVANT_DOCS''' içerisinde yer alan değerlere göre cevap ver. Verilen bilgilere göre cevaplayamadığın bir konuysa sadece '''UNK''' değeri ile cevap ver.
'''UNK''': 'Cevabı henüz bilmiyorum. Sana başka nasıl yardımcı olabilirim?'

'''RELEVANT_DOCS''':
3) tüm soruların için ve sigortaladığın ürünün hasarlanması durumunda [PHONE_NUMBER_MASK__0] numaralı trendyol sigorta müşteri hizmetleri'ne ulaşabilirsin.
trendyol sigorta iletişim bilgilerine nasıl ulaşabilirim?
tüm soru ve taleplerin için [PHONE_NUMBER_MASK__0] numaralı trendyol sigorta müşteri hizmetleri'ni

Lets check gpt response:

In [24]:
response["gpt_raw_response"]

"tüm soruların için ve sigortaladığın ürünün hasarlanması durumunda [PHONE_NUMBER_MASK__0] numaralı trendyol sigorta müşteri hizmetleri'ne ulaşabilirsin."

After post process:

In [25]:
response["parsed_response"]

"tüm soruların için ve sigortaladığın ürünün hasarlanması durumunda 0850 955 14 14 numaralı trendyol sigorta müşteri hizmetleri'ne ulaşabilirsin."

Also you can ask follow up questions:

In [26]:
response = chat("bu telefon numarasının haricinde mail adresi var mı?")



> Entering new LLMChain chain...
Prompt after formatting:
System: Bir eticaret firması için geliştirilmiş yardımcı ve nazik bir chatbotsun. 
'''RELEVANT_DOCS''' içerisindeki ifadelerin bazıları maskelenmiş olabilir. Cevap verirken bu alanları olduğu gibi bırak ve değişiklik yapma. Eğer maskelenmiş alan yoksa NONE değerini alır. Maskelenmiş alanlar şunlardan oluşuyor:
[EMAIL_MASK__0]
[PHONE_NUMBER_MASK__0]

Cevap verirken yorum yapma ve sadece '''RELEVANT_DOCS''' içerisinde yer alan değerlere göre cevap ver. Verilen bilgilere göre cevaplayamadığın bir konuysa sadece '''UNK''' değeri ile cevap ver.
'''UNK''': 'Cevabı henüz bilmiyorum. Sana başka nasıl yardımcı olabilirim?'

'''RELEVANT_DOCS''':
fax: [PHONE_NUMBER_MASK__0]

kep adresi: [EMAIL_MASK__0]
3. kargo seçiminizi yapın.

4. ekranda çıkan iade kargo kodunu not alın. iade kargo kodunuza siparişlerim sayfasından ve e-posta adresinize gönderilen bilgilendirme mesajından da ulaşabilirsiniz.
siparişimin teslimat adresini veya alıcı ad

In [27]:
response["gpt_raw_response"]

'Evet, trendyol sigorta ile iletişime geçmek için ayrıca [EMAIL_MASK__0] adresini de kullanabilirsin.'

In [28]:
response["parsed_response"]

'Evet, trendyol sigorta ile iletişime geçmek için ayrıca dsm@hs02.kep.tr adresini de kullanabilirsin.'